# Importations

In [9]:
import cv2
import tkinter
import torch
from PIL import Image
import os

import open3d as o3d
import numpy as np
import copy
import matplotlib
import matplotlib.pyplot as plt
import sys
import json 
import pandas as pd
from pyntcloud import PyntCloud
matplotlib.use('TkAgg')

import trimesh
from trimesh.transformations import transform_points
from io import BytesIO
from PIL import Image
import pyrender

from scipy.spatial import ConvexHull
from scipy.spatial.distance import cdist

# Test on room images

In [24]:
# Importation of the Yolo model
model = torch.hub.load(os.path.abspath('yolov5/'), 'custom', path=os.path.abspath('yolov5/yolov5s.pt'), source='local')  # local repo

YOLOv5  2022-9-21 Python-3.9.12 torch-1.12.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [25]:
# Importing test image
img = cv2.imread(os.path.abspath('open3d_data/room_right.png'))
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
img = cv2.circle(img, (700, 400), 20, (255, 0, 0), 2)

In [26]:
# Printing the results (boxes)

# Inference
results = model(img, size=1080)  # includes NMS

# Results
results.show()  
#results.show()  # or .show()

#results = results.xyxy[0]  # img1 predictions (tensor)
boxes = results.pandas().xyxy[0]  # img1 predictions (pandas)
boxes

,xmin,ymin,xmax,ymax,confidence,class,name
0,1005.970154,299.992401,1347.299194,746.199585,0.534302,28,suitcase
1,1008.942322,303.181671,1344.675903,742.474426,0.271587,56,chair


# Computation of the point on pixel image : sphere

<h3> Draw a line and the research of alpha

In [470]:
# Check if rotation is good

sphere_coordinate1 = [0,0,0]
mesh_sphere1 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere1.translate(sphere_coordinate1)


o3d.visualization.draw_geometries([pcd_room, mesh_sphere1])

In [385]:
sphere_coordinate = sphere_coordinate1

with open(os.path.abspath('open3d_data/room_test.json')) as json_file:
    data_camera = json.load(json_file)
extrinsic_matrix = np.array([[element for element in data_camera["extrinsic"][i:i+4]] for i in range(0,16,4)])
extrinsic_matrix = np.transpose(extrinsic_matrix)[0:3, :]
intrinsic_matrix = np.array([[element for element in data_camera["intrinsic"]["intrinsic_matrix"][i:i+3]] for i in range(0,9,3)])
intrinsic_matrix = np.transpose(intrinsic_matrix)
full_transfo_matrix = np.matmul(intrinsic_matrix, extrinsic_matrix)
array_3d_augmented = np.array(sphere_coordinate + [1])
array_2d_pixel = np.matmul(full_transfo_matrix, array_3d_augmented)

In [386]:
point1, point2 = tuple(int(x) for x in array_2d_pixel[0:2]//20), tuple(int(x) for x in array_2d_pixel[0:2]//5)
(x,y) = tuple(int(x) for x in array_2d_pixel[0:2]//array_2d_pixel[2])
image = cv2.imread(os.path.abspath('open3d_data/room_test.png'))
cv2.line(image, point1, point2, (255,0,0), 2)
cv2.circle(image, (x,y), radius=10, color=(0, 0, 255))
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [457]:
# Check if triangulation is good
pcd_room = o3d.io.read_point_cloud("C:/Users/raphaelanwarel.TRN/Documents/project/LIDAR/intro_O3D/data_test/fragment.ply")
center_chair_coor = space_coor
mesh_sphere_chair = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere_chair.translate(center_chair_coor)
o3d.visualization.draw_geometries([pcd_room, mesh_sphere_chair])

# Research of the 4 ground points with triangulation

In [29]:
# Functions to get Yolo Info 

def extract_biggest_confidence(boxes) :
    confidence_column = boxes["confidence"]
    number_of_index = len(boxes.index)
    if number_of_index > 1 :
        max_index = confidence_column.idxmax()
        return boxes.iloc[[max_index]]
    return boxes

def get_geometry_info(path, size=1080) :
    img = cv2.imread(path + ".png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )

    results = model(img, size=size)
    boxes = results.pandas().xyxy[0]  # img1 predictions (pandas
    chair_row = extract_biggest_confidence(boxes)
    center_box_pixel = [(chair_row["xmin"].to_numpy()[0] + chair_row["xmax"].to_numpy()[0])/2, (chair_row["ymin"].to_numpy()[0] + chair_row["ymax"].to_numpy()[0])/2]

    with open(path + ".json") as json_file:
        data_camera = json.load(json_file)

    extrinsic_matrix = np.array([[element for element in data_camera["extrinsic"][i:i+4]] for i in range(0,16,4)])
    extrinsic_matrix = np.transpose(extrinsic_matrix)[0:3, :]
    intrinsic_matrix = np.array([[element for element in data_camera["intrinsic"]["intrinsic_matrix"][i:i+3]] for i in range(0,9,3)])
    intrinsic_matrix = np.transpose(intrinsic_matrix)
    full_transfo_matrix = np.matmul(intrinsic_matrix, extrinsic_matrix)

    results = {"coordinate_center" : center_box_pixel, "extrinsic_matrix" : extrinsic_matrix, "intrinsic_matrix" : intrinsic_matrix, "camera_matrix" : full_transfo_matrix, 
                "xmin" : chair_row["xmin"].to_numpy()[0], "xmax" : chair_row["xmax"].to_numpy()[0], "ymin" : chair_row["ymin"].to_numpy()[0], "ymax" : chair_row["ymax"].to_numpy()[0]}
    return results

In [18]:
# Function to create the triangulation system

def create_system_matrix(results_1, results_2, focus="center") :
    g = {"results_1" : results_1, "results_2" : results_2}
    for j in range(1,3) :

        index_focus = 0

        if focus == "center" : #Take the center of the box
            g["x_" + str(j)] = g["results_" + str(j)]["coordinate_center"][0]
            g["y_" + str(j)] = g["results_" + str(j)]["coordinate_center"][1]

        if focus == "left_right" : # Put first the left picture (take the right bottom point)
            index_focus = j%2       # and then the right (bottom left point)
        elif focus == "right_left" :
            index_focus = (j + 1)%2

        if index_focus == 1 and focus != "center":
            g["x_" + str(j)] = g["results_" + str(j)]["xmax"]
            g["y_" + str(j)] = g["results_" + str(j)]["ymax"] 
        elif index_focus == 0 and focus != "center":  
            g["x_" + str(j)] = g["results_" + str(j)]["xmin"]
            g["y_" + str(j)] = g["results_" + str(j)]["ymax"]  

    
        for i in range(1,4) :
            g["p" + str(i) + "_" + str(j)] = g["results_" + str(j)]["camera_matrix"][i-1, :]

        
        g["row1_" + str(j)] = - np.multiply(g["x_" + str(j)], g["p3_" + str(j)]) + g["p1_" + str(j)]
        g["row2_" + str(j)] = np.multiply(g["y_" + str(j)], g["p3_" + str(j)]) - g["p2_" + str(j)]

        

    return np.matrix([g["row1_1"], g["row2_1"], g["row1_2"], g["row2_2"]])

In [521]:
# Apply these function to triangulate one point

path_img_left = os.path.abspath('open3d_data/room_left')
path_img_right = os.path.abspath('open3d_data/room_right')
model = torch.hub.load(os.path.abspath('yolov5/'), 'custom', path=os.path.abspath('yolov5/yolov5s.pt'), source='local')  

results_left = get_geometry_info(path_img_left)
results_right = get_geometry_info(path_img_right)

system_matrix = create_system_matrix(results_left, results_right, focus="left_right")
(u, s, vh) = np.linalg.svd(system_matrix)
space_coor = vh[-1, :]
space_coor = np.array([space_coor[0,i] for i in range(4)])
space_coor = (space_coor/space_coor[-1])[:3]


YOLOv5  2022-9-21 Python-3.9.12 torch-1.12.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [523]:
# Check if triangulation is good
pcd_room = o3d.io.read_point_cloud("C:/Users/raphaelanwarel.TRN/Documents/project/LIDAR/intro_O3D/data_test/fragment.ply")
center_chair_coor = space_coor
mesh_sphere_chair = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere_chair.translate(center_chair_coor)
o3d.visualization.draw_geometries([pcd_room, mesh_sphere_chair])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


# Replacing a chair by something else : Rotating and croping the room

In [10]:
# Load the room
vol = o3d.visualization.read_selection_polygon_volume("C:/Users/raphaelanwarel.TRN/Documents/project/LIDAR/intro_O3D/data_test/cropped.json")
pcd_room = o3d.io.read_point_cloud("C:/Users/raphaelanwarel.TRN/Documents/project/LIDAR/intro_O3D/data_test/fragment.ply")

#o3d.visualization.draw_geometries([pcd_room], width=1400, height=900)


<h3> Rotating and translating the room

In [164]:
# Rotation and translation function
index_ground = 191570

def rotation_toward_axis(mesh, initial_orientation, final_orientation=np.array([0, 0, 1])) :

    ### Make the mesh going in the direction (0, 0, 1)
    norm = np.linalg.norm(initial_orientation)
    initial_orientation = initial_orientation / norm
    norm = np.linalg.norm(final_orientation)
    final_orientation = final_orientation / norm
       
    # Computation of Rotation matrix
    v = np.cross(initial_orientation, final_orientation)
    s, c = np.linalg.norm(v), np.dot(initial_orientation, final_orientation)

    if c != -1 :
        v_bracket = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        R = np.eye(3) + v_bracket + np.matmul(v_bracket, v_bracket) * (1 - c)/s**2

    else :
        R = -np.eye(3)
        R[1,1] = 1
    
    # Transformation Matrix
    T = np.eye(4)
    T[0:3, 0:3] = R
    if np.linalg.norm(initial_orientation) != 0:
        mesh = mesh.transform(T)


def ground_to_zero(mesh, z_ground_coordinate) :
    # Put the groud at z=0 after rotation
    T = np.eye(4)
    T[2, 3] = - z_ground_coordinate
    mesh = mesh.transform(T)

In [12]:
# Rotation and translation of the room
pcd_room.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.1, max_nn=100))
initial_orientation = pcd_room.normals[index_ground]
rotation_toward_axis(pcd_room, initial_orientation)

z_ground_coordinate = np.asarray(pcd_room.points)[index_ground][2]
ground_to_zero(pcd_room, z_ground_coordinate)

In [13]:
# Check if rotation is good
sphere_coordinate1 = [0,0,0]
mesh_sphere1 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere1.translate(sphere_coordinate1)

sphere_coordinate2 = [0,0,1]
mesh_sphere2 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere2.translate(sphere_coordinate2)

o3d.visualization.draw_geometries([pcd_room,mesh_sphere1, mesh_sphere2])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


<h3> Finding 4 good points and croping : naive method

In [204]:
# Interactive function to select points
def pick_points(pcd):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) Afther picking points, press q for close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

pick_points(pcd_room)

''' 
[Open3D INFO] Picked point #194040 (2.0, 1.7, -2.4) to add in queue.
[Open3D INFO] Picked point #156031 (2.2, 0.8, -2.4) to add in queue.
[Open3D INFO] Picked point #162899 (2.8, 0.81, -2.4) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #193799 (2.7, 1.7, -2.4) to add in queue.
'''


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) Afther picking points, press q for close the window



' \n[Open3D INFO] Picked point #194040 (2.0, 1.7, -2.4) to add in queue.\n[Open3D INFO] Picked point #156031 (2.2, 0.8, -2.4) to add in queue.\n[Open3D INFO] Picked point #162899 (2.8, 0.81, -2.4) to add in queue.\n[Open3D INFO] No point has been picked.\n[Open3D INFO] Picked point #193799 (2.7, 1.7, -2.4) to add in queue.\n'

In [287]:
# Index of 4 interesting points
index1 = 194040
index2 = 156031
index3 = 162899
index4 = 193799

In [288]:
# Cropping of the room and the old chair
delta_z = 2
epsilon_z = delta_z /80


corners = np.array([ 
    np.asarray(pcd_room.points)[index1] + np.array([0, 0, epsilon_z]),
    np.asarray(pcd_room.points)[index2] + np.array([0, 0, epsilon_z]),
    np.asarray(pcd_room.points)[index3] + np.array([0, 0, epsilon_z]),
    np.asarray(pcd_room.points)[index4] + np.array([0, 0, epsilon_z]),
    np.asarray(pcd_room.points)[index1] + np.array([0, 0, delta_z]),
    np.asarray(pcd_room.points)[index2] + np.array([0, 0, delta_z]),
    np.asarray(pcd_room.points)[index3] + np.array([0, 0, delta_z]),
    np.asarray(pcd_room.points)[index4] + np.array([0, 0, delta_z]),
])


cuboid_points = corners
points = o3d.utility.Vector3dVector(cuboid_points)
oriented_bounding_box = o3d.geometry.OrientedBoundingBox.create_from_points(points)
old_chair = pcd_room.crop(oriented_bounding_box)

dists = pcd_room.compute_point_cloud_distance(old_chair)
dists = np.asarray(dists)
ind = np.where(dists > .01)[0]
pcd_without_chair = pcd_room.select_by_index(ind)

#o3d.visualization.draw_geometries([pcd_without_chair, oriented_bounding_box])


<h3> Finding 4 good points and croping : triangulation

In [30]:
# Find the center and one surrounding point (not in the pointcloud) + finding the symmetric and all the other points

correction_coef = 0.8 # Percentage of distance reduction between center and surrounding points

path_img_left = os.path.abspath('open3d_data/room_left')
path_img_right = os.path.abspath('open3d_data/room_right')
model = torch.hub.load(os.path.abspath('yolov5/'), 'custom', path=os.path.abspath('yolov5/yolov5s.pt'), source='local')  

results_left = get_geometry_info(path_img_left)
results_right = get_geometry_info(path_img_right)

system_matrix = create_system_matrix(results_left, results_right, focus="center")
(u, s, vh) = np.linalg.svd(system_matrix)
space_coor_center = vh[-1, :]
space_coor_center = np.array([space_coor_center[0,i] for i in range(4)])
space_coor_center = (space_coor_center/space_coor_center[-1])[:3]

system_matrix = create_system_matrix(results_left, results_right, focus="left_right")
(u, s, vh) = np.linalg.svd(system_matrix)
space_coor_out_1 = vh[-1, :]
space_coor_out_1 = np.array([space_coor_out_1[0,i] for i in range(4)])
space_coor_out_1 = (space_coor_out_1/space_coor_out_1[-1])[:3]
space_coor_out_1[2] = 0

## Correction of space_coor_out_1
space_coor_out_1 = space_coor_center + correction_coef*(space_coor_out_1 - space_coor_center)


# Symmetrical points
space_coor_out_3 = space_coor_center + (space_coor_center - space_coor_out_1)
space_coor_out_3[2] = 0

angle = np.pi/2
rotation_matrix = np.mat([[np.cos(angle), -np.sin(angle), 0], [np.sin(angle), np.cos(angle), 0], [0, 0, 0]])

space_coor_out_2 = space_coor_center + np.matmul(rotation_matrix, space_coor_out_1 - space_coor_center)
space_coor_out_2 = np.array([space_coor_out_2[0,i] for i in range(3)])
space_coor_out_2[2] = 0

space_coor_out_4 = space_coor_center + np.matmul(rotation_matrix, space_coor_out_3 - space_coor_center)
space_coor_out_4 = np.array([space_coor_out_4[0,i] for i in range(3)])
space_coor_out_4[2] = 0

YOLOv5  2022-9-21 Python-3.9.12 torch-1.12.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


{'results_1': {'coordinate_center': [1169.3529968261719, 577.9370574951172], 'extrinsic_matrix': array([[    0.90972,    -0.41502,   -0.012811,     -1.1662],
       [  -0.066453,    -0.11507,    -0.99113,      0.7878],
       [    0.40986,     0.90251,    -0.13226,     0.16613]]), 'intrinsic_matrix': array([[     935.31,           0,       959.5],
       [          0,      935.31,       539.5],
       [          0,           0,           1]]), 'camera_matrix': array([[     1244.1,      477.78,     -138.89,     -931.36],
       [     158.97,      379.28,     -998.37,      826.46],
       [    0.40986,     0.90251,    -0.13226,     0.16613]]), 'xmin': 985.8502807617188, 'xmax': 1352.855712890625, 'ymin': 387.5121154785156, 'ymax': 768.3619995117188}, 'results_2': {'coordinate_center': [1176.6007385253906, 523.0942840576172], 'extrinsic_matrix': array([[    0.94314,     0.28759,    -0.16665,     -2.1393],
       [  -0.064604,    -0.33321,    -0.94064,     0.97319],
       [   -0.32604,   

In [31]:
# Check if triangulation is good (out of pointcloud)
center_chair_coor = space_coor_center
mesh_sphere_chair = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere_chair.translate(center_chair_coor)

out_chair_coor_1 = space_coor_out_1
mesh_out_1 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_out_1.translate(space_coor_out_1)

out_chair_coor_3 = space_coor_out_3
mesh_out_3 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_out_3.translate(space_coor_out_3)

out_chair_coor_2 = space_coor_out_2
mesh_out_2 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_out_2.translate(space_coor_out_2)

out_chair_coor_4 = space_coor_out_4
mesh_out_4 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_out_4.translate(space_coor_out_4)

o3d.visualization.draw_geometries([pcd_room, mesh_sphere_chair, mesh_out_1, mesh_out_3, mesh_out_2, mesh_out_4])

In [32]:
# Function to find the nearest point (in the pointcloud) of these points

def find_nearest(pcd, point):
    pcd_points = np.asarray(pcd.points)
    idx = (np.linalg.norm(pcd_points - point, axis=1)).argmin()
    return idx


In [68]:
# Apply function to get the 4 index

index1_tri = find_nearest(pcd_room, space_coor_out_1)
index2_tri = find_nearest(pcd_room, space_coor_out_2)
index3_tri = find_nearest(pcd_room, space_coor_out_3)
index4_tri = find_nearest(pcd_room, space_coor_out_4)

In [69]:
# Cropping of the room and the old chair
delta_z = 2
epsilon_z = delta_z /80


corners = np.array([ 
    np.asarray(pcd_room.points)[index1_tri] + np.array([0, 0, epsilon_z]),
    np.asarray(pcd_room.points)[index2_tri] + np.array([0, 0, epsilon_z]),
    np.asarray(pcd_room.points)[index3_tri] + np.array([0, 0, epsilon_z]),
    np.asarray(pcd_room.points)[index4_tri] + np.array([0, 0, epsilon_z]),
    np.asarray(pcd_room.points)[index1_tri] + np.array([0, 0, delta_z]),
    np.asarray(pcd_room.points)[index2_tri] + np.array([0, 0, delta_z]),
    np.asarray(pcd_room.points)[index3_tri] + np.array([0, 0, delta_z]),
    np.asarray(pcd_room.points)[index4_tri] + np.array([0, 0, delta_z]),
])


cuboid_points = corners
points = o3d.utility.Vector3dVector(cuboid_points)
oriented_bounding_box = o3d.geometry.OrientedBoundingBox.create_from_points(points)
old_chair = pcd_room.crop(oriented_bounding_box)

dists = pcd_room.compute_point_cloud_distance(old_chair)
dists = np.asarray(dists)
ind = np.where(dists > .01)[0]
pcd_without_chair = pcd_room.select_by_index(ind)

o3d.visualization.draw_geometries([pcd_without_chair, oriented_bounding_box])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


<h3> Creating a clean old chair

In [35]:
# Function to clean the old chair
def cleaning_cropped_object(mesh) :

    # Detect the biggest object in the point cloud
    # Create 2 differents points cloud : one with the big object, and one with the rest
    mesh_initial = copy.deepcopy(mesh)
    with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            mesh.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))

    biggest_label = np.unique(labels, return_counts=True)[0][np.unique(labels, return_counts=True)[1].argmax()]
    big_object_points, big_object_colors, artefacts_points, artefacts_colors = [], [], [], []
    
    for i in range(len(labels)) :
        if labels[i] == biggest_label :
            big_object_points.append(np.asarray(mesh_initial.points)[i])
            big_object_colors.append(np.asarray(mesh_initial.colors)[i])
        else :
            artefacts_points.append(np.asarray(mesh_initial.points)[i])
            artefacts_colors.append(np.asarray(mesh_initial.colors)[i])

    big_object, artefacts = o3d.geometry.PointCloud(), o3d.geometry.PointCloud()
    big_object.points, artefacts.points = o3d.utility.Vector3dVector(np.array(big_object_points)), o3d.utility.Vector3dVector(np.array(artefacts_points))
    big_object.colors, artefacts.colors = o3d.utility.Vector3dVector(np.array(big_object_colors)), o3d.utility.Vector3dVector(np.array(artefacts_colors))
    del mesh_initial

    return big_object, artefacts


In [48]:
# Cleaning : Old chair and artefacts
cleaned_old_chair, artefacts = cleaning_cropped_object(old_chair)
artefacts, ind = artefacts.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=2.0)
#o3d.visualization.draw_geometries([pcd_without_chair])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 4


<h3> Merging artefact with the cropped room

In [49]:
# Creation of the cleaned cropped room
pcd_without_chair_cleaned = o3d.geometry.PointCloud()
pcd_without_chair_cleaned.points = o3d.utility.Vector3dVector(np.concatenate((np.asarray(pcd_without_chair.points), np.asarray(artefacts.points)), axis=0))
pcd_without_chair_cleaned.colors = o3d.utility.Vector3dVector(np.concatenate((np.asarray(pcd_without_chair.colors), np.asarray(artefacts.colors)), axis=0))
#o3d.visualization.draw_geometries([pcd_without_chair_cleaned])

# Rotate, Scale and move the new chair

<h3> Importation of new chair and downsample if needed

In [175]:
# Load new chair and downsample
new_chair = o3d.io.read_point_cloud(os.path.abspath('open3d_data/chair/wooden_chair.ply'))

while len(new_chair.points) > len(cleaned_old_chair.points) :
    new_chair = new_chair.uniform_down_sample(2)
    
new_chair.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.1, max_nn=100))
o3d.visualization.draw_geometries([new_chair], width=1400, height=900)

PointCloud with 15611 points.


In [165]:
# Load new chair and downsample
new_chair = o3d.io.read_point_cloud(os.path.abspath('open3d_data/chair/Patchwork_chair.ply'))

while len(new_chair.points) > len(cleaned_old_chair.points) :
    new_chair = new_chair.uniform_down_sample(2)

new_chair.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.1, max_nn=100))
#o3d.visualization.draw_geometries([new_chair], width=1400, height=900)

<h3> Find the good rotation : Naive method (Point selection on new chair)



In [176]:
# Picking a point

def pick_points(pcd):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) Afther picking points, press q for close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

new_chair.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.1, max_nn=20))
pick_points(new_chair)
"""  
Picked point #11002 (-0.054, 0.036, -0.83) to add in queue. For the patchwork chair
Picked point #14500 (-6.9, -61., -17.) to add in queue. For the wooden chair
"""


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) Afther picking points, press q for close the window
[Open3D INFO] Picked point #14500 (-6.9, -61., -17.) to add in queue.



'  \nPicked point #11002 (-0.054, 0.036, -0.83) to add in queue. For the patchwork chair\n'

In [179]:
# Rotation of the chair toward z
good_index_new_chair_patch = 11002
good_index_new_chair = 14500
initial_orientation = np.asarray(new_chair.normals[good_index_new_chair]) #Normals are buggy with the chair
rotation_toward_axis(new_chair, initial_orientation)

In [180]:
# Checking direction of the chair

sphere_coordinate1 = [0,0,0]
mesh_sphere1 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere1.translate(sphere_coordinate1)

sphere_coordinate2 = [0,0,1]
mesh_sphere2 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere2.translate(sphere_coordinate2)

o3d.visualization.draw_geometries([pcd_without_chair_cleaned, new_chair, mesh_sphere1, mesh_sphere2])

<h3> Find the good z translation

In [181]:
# Functions translation chair

def translation_chair_z(chair_mesh):
    # Find the lowest point of the rotated chair
    # Translation to put this point at z=0
    lowest_z = np.min(np.asarray(chair_mesh.points)[:, 2])
    T = np.eye(4)
    T[2, 3] = - lowest_z
    chair_mesh = chair_mesh.transform(T)

In [182]:
# Apply translation transformation
translation_chair_z(new_chair)

In [183]:
# Checking direction of the chair

sphere_coordinate1 = [0,0,0]
mesh_sphere1 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere1.translate(sphere_coordinate1)

sphere_coordinate2 = [0,0,1]
mesh_sphere2 = o3d.geometry.TriangleMesh.create_sphere(radius = .1)
mesh_sphere2.translate(sphere_coordinate2)

o3d.visualization.draw_geometries([pcd_without_chair_cleaned, new_chair, mesh_sphere1, mesh_sphere2])

<h3> Same direction as the old chair

In [189]:
# Computing mean value for bottom upper part, fetching on x and y coordinate 

height_old_chair = np.max(np.asarray(cleaned_old_chair.points)[:, 2])
height_new_chair = np.max(np.asarray(new_chair.points)[:, 2]) # The ground is at z=0

cleaned_old_chair.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.1, max_nn=100))
new_chair.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.1*height_new_chair/height_old_chair, max_nn=100))

n = len(np.asarray(new_chair.normals))
direction_new_chair = np.mean(np.array([np.asarray(new_chair.normals)[i] for i in range(n) if np.asarray(new_chair.points)[i][2] >= height_new_chair/2]), axis=0)
direction_new_chair[2] = 0

n = len(np.asarray(cleaned_old_chair.normals))
direction_old_chair = np.mean(np.array([np.asarray(cleaned_old_chair.normals)[i] for i in range(n) if np.asarray(cleaned_old_chair.points)[i][2] >= height_old_chair/2]), axis=0)
direction_old_chair[2] = 0

rotation_toward_axis(new_chair, initial_orientation=direction_new_chair, final_orientation=direction_old_chair)

o3d.visualization.draw_geometries([new_chair, cleaned_old_chair])

<h3> Find the good scaling and translation : fit in the bounding box

In [ ]:
# Old method
def max_distance(points) :
    hull = ConvexHull(points)
    # Extract the points forming the hull
    hullpoints = points[hull.vertices,:]
    hdist = cdist(hullpoints, hullpoints, metric='euclidean')
    return np.max(hdist)

# Transform : same height for both chairs
old_height = max_distance(np.asarray(old_chair.points))
new_height = max_distance(np.asarray(new_chair.points))

scale_coef = old_height / new_height

# Transformation Matrix
T = scale_coef*np.eye(4)
T[3, 3] = 1
new_chair = new_chair.transform(T)

# Scale and translate in the bounding box
index1 = 194040
index2 = 156031
index3 = 162899
index4 = 193799
good_index_new_chair = 11002  #Index of a point in the middle of the new chair
list_index = [index1, index2, index3, index4]
scaling_in_bounding_box(chair_mesh=new_chair, room_mesh=pcd_room, list_index=list_index)
translating_in_bounding_box(chair_mesh=new_chair, room_mesh=pcd_room, list_index=list_index, index_middle_chair=good_index_new_chair) 

In [190]:
# Functions of scaling and translation

def scaling_in_bounding_box(chair_mesh, room_mesh, list_index) :

    # Fit the chair in the box
    chair_points = np.asarray(chair_mesh.points)
    room_points = np.asarray(room_mesh.points)

    hull = ConvexHull(chair_points)
    # Extract the points forming the hull (only x and y coordinate)
    hullpoints = chair_points[hull.vertices,0:2]
    hdist = cdist(hullpoints, hullpoints, metric='euclidean')
    diameter_chair = np.max(hdist)

    # Find the smallest distance between two vertices
    hullpoints_box = room_points[list_index,0:2]
    hdist = cdist(hullpoints_box, hullpoints_box, metric='euclidean')
    diameter_box = np.min(hdist[np.nonzero(hdist)])


    scale_coef = diameter_box / diameter_chair

    # Transformation Matrix
    T = scale_coef*np.eye(4)
    T[3, 3] = 1
    chair_mesh = chair_mesh.transform(T)

def translating_in_bounding_box(chair_mesh, room_mesh, list_index, index_middle_chair) :
    
    # Move the middle point of the chair to the box barycenter
    vertices_xy = np.asarray(room_mesh.points)[list_index,0:2]
    box_barycenter = np.sum(vertices_xy, axis=0)/4
    chair_barycenter = np.asarray(chair_mesh.points)[index_middle_chair,0:2]
    translation = box_barycenter - chair_barycenter

    T = np.eye(4)
    T[0:2, 3] = translation
    chair_mesh = chair_mesh.transform(T)

def translating_center_chair(new_chair_mesh, old_chair_mesh, list_index, index_middle_chair) :
    
    # Move the middle point of the chair to the box barycenter
    vertices_xy = np.asarray(old_chair_mesh.points)[list_index,0:2]
    old_chair_barycenter = np.sum(vertices_xy, axis=0)/4
    new_chair_barycenter = np.asarray(new_chair_mesh.points)[index_middle_chair,0:2]
    translation = old_chair_barycenter - new_chair_barycenter

    T = np.eye(4)
    T[0:2, 3] = translation
    new_chair_mesh = new_chair_mesh.transform(T)




In [191]:
# Function to find a bounding box for cleaned old_chair

def surrounding_box(pcd) :

    points = np.asarray(pcd.points)
    index1 = np.argmax(points[:, 0])
    index2 = np.argmax(points[:, 1])
    index3 = np.argmin(points[:, 0])
    index4 = np.argmin(points[:, 1])
    return [index1, index2, index3, index4]

In [192]:
# Scale and translate in the bounding box

good_index_new_chair = 11002  #Index of a point in the middle of the new chair
list_index = surrounding_box(cleaned_old_chair)
scaling_in_bounding_box(chair_mesh=new_chair, room_mesh=pcd_room, list_index=list_index)
translating_center_chair(new_chair_mesh=new_chair, old_chair_mesh=cleaned_old_chair, list_index=list_index, index_middle_chair=good_index_new_chair)
o3d.visualization.draw_geometries([pcd_without_chair_cleaned, new_chair, mesh_sphere1, mesh_sphere2])